In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [4]:
import gradio as gr
import torch, numpy as np, cv2, math
from transformers import CLIPProcessor, CLIPModel
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from PIL import Image

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"


# Load Models (once at startup)

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
room_prompts = {
    "bedroom": "A cozy and modern bedroom interior with warm lighting.",
    "living room": "A spacious and stylish living room interior with contemporary decor.",
    "kitchen": "A modern kitchen interior with state-of-the-art appliances and clean lines.",
    "office": "A professional office interior with ergonomic furniture and minimalist design.",
    "bathroom": "A luxurious bathroom interior with elegant fixtures and soothing colors."
}

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_canny",
    torch_dtype=torch.float16 if device=="cuda" else torch.float32
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32
).to(device)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

tokenizer%2Ftokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vae%2Fconfig.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

unet%2Fconfig.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
def detect_features(image: Image.Image):
    # List of common room features
    features_list = ["window", "sofa", "table", "lamp", "bookshelf", "artwork", "rug", "plant", "door", "curtains"]
    text_prompts = [f"A photo of a {feat}" for feat in features_list]
    inputs = clip_processor(text=text_prompts, images=image, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = clip_model(**inputs)
        probs = outputs.logits_per_image.softmax(dim=1).cpu().numpy()[0]
    # Select the top 3 features based on probabilities
    top_indices = np.argsort(probs)[-3:][::-1]
    detected = [features_list[i] for i in top_indices]
    return detected



In [6]:

# Function: Generate Designs Based on Room Analysis

def generate_design(input_image: Image.Image, variant_count: int):
    if input_image is None:
        return None, None, None
    room_image = input_image.convert("RGB")

    # Room type detection using CLIP
    text_inputs = list(room_prompts.values())
    inputs = clip_processor(text=text_inputs, images=room_image, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = clip_model(**inputs)
        probs = outputs.logits_per_image.softmax(dim=1).cpu().numpy()[0]
    detected_room_type = list(room_prompts.keys())[int(np.argmax(probs))]

    # Detect perspective using Canny edges
    room_np = np.array(room_image)
    gray = cv2.cvtColor(room_np, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)
    angles = []
    if lines is not None:
        for x1, y1, x2, y2 in lines[:, 0]:
            angles.append(np.arctan2((y2 - y1), (x2 - x1)) * (180 / np.pi))
    deviations = [min(abs(a), abs(a-90), abs(a+90)) for a in angles] if angles else [0]
    angled = (np.mean(deviations) > 10)
    perspective = ("shot from an angled perspective, with dynamic depth cues and natural distortions"
                   if angled else "in a clear, frontal view")

    # Detect additional room features
    detected_features = detect_features(room_image)
    features_text = ", ".join(detected_features)

    # Build a comprehensive, ultra-advanced prompt
    prompt = (f"Design a superlative interior for a {detected_room_type} that is ultra-modern, luxurious, and breathtaking. "
              f"Key room features detected: {features_text}. Emphasize exquisite details, creative lighting, and opulent decor. "
              f"Design optimized for a room {perspective}.")

    # Create conditioning image (using Canny edges)
    conditioning_image = Image.fromarray(np.stack([edges]*3, axis=-1))

    # Generate multiple design variants
    if device == "cuda":
        with torch.autocast("cuda"):
            outputs = pipe(prompt, image=conditioning_image, num_inference_steps=50, num_images_per_prompt=variant_count)
    else:
        outputs = pipe(prompt, image=conditioning_image, num_inference_steps=50, num_images_per_prompt=variant_count)
    generated_images = outputs.images

    return generated_images, prompt, room_image



In [7]:
# Function: Refine Designs Based on Feedback

def refine_design(feedback_text, stored_prompt, stored_input_image: Image.Image, variant_count: int):
    if stored_input_image is None or stored_prompt is None:
        return None, stored_prompt, stored_input_image
    # Append feedback to the stored prompt for refinement
    new_prompt = stored_prompt + " " + feedback_text
    room_image = stored_input_image.convert("RGB")
    room_np = np.array(room_image)
    gray = cv2.cvtColor(room_np, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    conditioning_image = Image.fromarray(np.stack([edges]*3, axis=-1))

    if device == "cuda":
        with torch.autocast("cuda"):
            outputs = pipe(new_prompt, image=conditioning_image, num_inference_steps=50, num_images_per_prompt=variant_count)
    else:
        outputs = pipe(new_prompt, image=conditioning_image, num_inference_steps=50, num_images_per_prompt=variant_count)
    generated_images = outputs.images
    return generated_images, new_prompt, stored_input_image



In [8]:
# ---------------------------
# Gradio Interface with Iterative Feedback Loop
# ---------------------------
with gr.Blocks() as demo:
    gr.Markdown("## Super Advanced Interior Design Transformer with Feature Detection")
    with gr.Row():
        with gr.Column(scale=1):
            input_image = gr.Image(type="pil", label="Upload Your Room Image")
            variant_count = gr.Slider(1, 5, value=3, step=1, label="Number of Variants")
            generate_btn = gr.Button("Generate All Possible Interior Designs")
        with gr.Column(scale=1):
            gallery = gr.Gallery(label="Generated Interior Designs", show_label=True)

    # Hidden states to store the latest prompt and input image
    stored_prompt = gr.State()
    stored_input = gr.State()

    generate_btn.click(fn=generate_design, inputs=[input_image, variant_count],
                       outputs=[gallery, stored_prompt, stored_input])

    gr.Markdown("### Provide Feedback to Update the Designs")
    with gr.Row():
        feedback_text = gr.Textbox(label="Feedback on Design", placeholder="e.g., 'increase brightness, add more luxury details'...")
        refine_btn = gr.Button("Refine Designs")

    refined_gallery = gr.Gallery(label="Refined Interior Designs", show_label=True)
    refine_btn.click(fn=refine_design, inputs=[feedback_text, stored_prompt, stored_input, variant_count],
                     outputs=[refined_gallery, stored_prompt, stored_input])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f21197d77a3e907990.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
